In [ ]:
# import pandas as pd
#  import matplotlib.pyplot as plt
from activation_functions import softmax
# from tensorflow.keras import activations
tf_sigmoid = activations.sigmoid
def sigmoid(X):
    return tf_sigmoid(X).numpy()

import numpy as np
import pandas as pd
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import re
from sklearn.metrics import mean_squared_error
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# from tensorflow.keras.losses import CategoricalCrossentropy




In [380]:
class MY_LSTM:
    
    @staticmethod
    def my_build_model(hidden_size, input_size):
        lstm = MY_LSTM(hidden_size = hidden_size, input_size = input_size, optimizer=0)
        return lstm

    
    def __init__(self, hidden_size, input_size, optimizer):
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = 1
        self._optimizer = self.Adam
        self._loss_func = mean_squared_error
        self.USE_OPTIMIZER = True


        # input_size = 2
        # hidden_size = 1
        # z_size = 24
        z_size = self.hidden_size + self.input_size
        
        
        # https://github.com/keras-team/keras/blob/v2.12.0/keras/layers/rnn/lstm.py
        # line 274
        self.parameters = {
            #"weights" is for new information input
            #"recurrent" is for hidden_state(cell memory) passed to lstm from prev lstm (ht-1)
            "weights": {
                "Forget": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Input": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Candidate": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "Output": self.__init_orthogonal(np.random.randn(z_size, self.hidden_size)),
                "OutputSoftmax": self.__init_orthogonal(np.random.randn(self.hidden_size, self.output_size)),
            },
            "recurrent": {
                "Forget": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Input": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Candidate": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size)),
                "Output": self.__init_orthogonal(np.random.randn(self.hidden_size, self.hidden_size))
            },
            "bias": {
                "Forget": np.zeros((1, self.hidden_size)),
                "Input": np.zeros((1, self.hidden_size)),
                "Candidate": np.zeros((1, self.hidden_size)),
                "Output": np.zeros((1, self.hidden_size)),
                "OutputSoftmax": np.zeros((1, self.output_size)),

            }
        }

    @property
    def optimizer(self):
        return self._optimizer

    @optimizer.setter
    def optimizer(self, optimizer):
        self._optimizer = optimizer


    @property
    def loss_func(self):
        return self._loss_func

    @loss_func.setter
    def loss_func(self, loss_func):
        self.loss_func = loss_func


    def __init_orthogonal(self, param):
        """
        Initializes weight parameters orthogonally.
        This is a common initiailization for recurrent neural networks.

        Refer to this paper for an explanation of this initialization:
            https://arxiv.org/abs/1312.6120
        """
        if param.ndim < 2:
            raise ValueError("Only parameters with 2 or more dimensions are supported.")

        rows, cols = param.shape

        new_param = np.random.randn(rows, cols)

        if rows < cols:
            new_param = new_param.T

        # Compute QR factorization
        q, r = np.linalg.qr(new_param)

        d = np.diag(r, 0)
        ph = np.sign(d)
        q *= ph

        if rows < cols:
            q = q.T

        new_param = q

        return new_param

    @property
    def get_parameters(self):
        """
        Returns weights and biases as 2d array

        """
        return self.parameters

    def forward(self, input_data, prev_stm, prev_ltm):
        """
        Arguments:
            input_data -- your input data at timestep "t", numpy array of shape (n_x, m).
            prev_stm -- h at timestep "t-1", numpy array of shape (n_a, m)
            prev_ltm -- c at timestep "t-1", numpy array of shape (n_a, m)
        Returns:
            outputs -- prediction at timestep "t", numpy array of shape (n_v, m)
            
            Weight shape:
            All of the weights    shape (self.hidden_size, z_size)
            Except W_OutputSoftmax      (self.hidden_size, self.output_size)
            
            Data shapes:
            input_data            shape (1,1)
            prev_stm and prev_ltm shape (1,hidden_state)
            concat                shape is (1, z_size)
            concat.T              shape is (z_size, 1)
             - forget_gate        shape is (hidden_size, 1)
             - input_gate         shape is (hidden_size, 1)
             - candidate          shape is (hidden_size, 1)
             - next_ltm           shape is (1, hidden_size)
        """

        # Save a list of computations for each of the components in the LSTM
        
        concat = np.concatenate((prev_stm, input_data), axis=1)
#         print(f"Concat value is: \n", concat, "\nConcat.T value is: \n", concat.T)
        
#         Compute the forget gate
#         print("=======Computation of the forget gate:======")
#         print(f"W_forget:\n {self.parameters['weights']['Forget']}\nconcat:\n {concat.T}\nb_Forget: {self.parameters['bias']['Forget']}")
        z = np.dot(prev_stm, self.parameters['recurrent']['Forget'])
        y = np.dot(concat, self.parameters['weights']['Forget'])
#         print(f"np.dot(stm, forget): {z}")
#         print(f"np.dot(concat, forget): {y}")
#         print(f"bias: {self.parameters['bias']['Forget']}")
        
        forget_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Forget"]) + np.dot(prev_stm, self.parameters["recurrent"]["Forget"]) + self.parameters["bias"]["Forget"])
#         print(f"Forget_gate value is: \n", forget_gate)
#         print("===========================================")


        # Compute the input gate
#         print(f"input_gate = concate.shape is {concat.shape} dot W_Input shape is {self.parameters['weights']['Input'].shape}")
        input_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Input"]) + np.dot(prev_stm, self.parameters["recurrent"]["Input"]) + self.parameters["bias"]["Input"])
#         print(f"Input_gate value is: \n", input_gate)

        # Compute the candidate cell value
        candidate = np.tanh(np.dot(concat, self.parameters["weights"]["Candidate"]) + np.dot(prev_stm, self.parameters["recurrent"]["Candidate"]) + self.parameters["bias"]["Candidate"])
#         print(f"Candidate_gate value is: \n", candidate)

        # Compute the memory cell
#         print(f"==============next_ltm===========\n")
#         print(f"forget_gate * prev_ltm:\n {forget_gate * prev_ltm}")
#         print(f"input_gate * candidate: \n {input_gate * candidate}")
#         print(f"next_ltm:{forget_gate * prev_ltm + input_gate * candidate} ")
        next_ltm = forget_gate * prev_ltm + input_gate * candidate
#         print(f"next_ltm value is: \n", next_ltm)
#         print("==================================")


        # Compute the output gate
        output_gate = sigmoid(np.dot(concat, self.parameters["weights"]["Output"]) + np.dot(prev_stm, self.parameters["recurrent"]["Output"]) + self.parameters["bias"]["Output"])
#         print(f"Output_gate value is: \n", output_gate)

        # Compute the next hidden state
        next_stm = output_gate * np.tanh(next_ltm)
#         print(f"next_stm value is: \n", next_stm)

        forward_pass = {
            "Forget": forget_gate,
            "Input": input_gate,
            "Candidate": candidate,
            "Output": output_gate,
            "next_ltm": next_ltm,
            "next_stm": next_stm,
            "Concat_Input": concat
        }
        
        return forward_pass

    def __cross_entropy(self, predictions, targets, epsilon=1e-12):
        """
        Input: predictions (N, 1) ndarray
               targets (N, 1) ndarray
        Returns: scalar
        """
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        N = predictions.shape[0]
        ce = -np.sum(targets * np.log(predictions + epsilon)) / N
        return ce
    
    def cross_entropy_loss(y_pred, y_true, epsilon=1e-10):
        y_pred = np.clip(y_pred, epsilon, 1.0 - epsilon)
        num_samples = y_pred.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + epsilon)) / num_samples
        return loss

    def calculate_loss(self, prediction, targets):
#         print(f"============IN LOSS=============")
#         print(f"prediction: {prediction}")
#         print(f"targets: {targets}")
        targets = [[targets]]
        x = self.loss_func(prediction, targets)
#         print(x)
        return x

    def backward(self, forward_pass, prediction, targets):
        """
        Arguments:
        forward_pass -- dictionary:
                        "Forget_gate": forget_gate,
                        "Input_gate": input_gate,
                        "Candidate_gate": candidate,
                        "Output_gate": output_gate,
                        "next_ltm": next_ltm.T,
                        "next_stm": next_stm.T
                        
        targets -- your targets as a list of size m.
        Returns:
        loss -- crossentropy loss for all elements in output
        grads -- lists of gradients of every element in p
        
        Weight shape:
            All of the weights    shape (self.hidden_size, z_size)
            Except OutputSoftmax      (self.hidden_size, self.output_size)

        """

        gradients = {
            "weights": {
                "Forget": np.zeros_like(self.parameters["weights"]["Forget"]),
                "Input": np.zeros_like(self.parameters["weights"]["Input"]),
                "Candidate": np.zeros_like(self.parameters["weights"]["Candidate"]),
                "Output": np.zeros_like(self.parameters["weights"]["Output"]),
                "OutputSoftmax": np.zeros_like(self.parameters["weights"]["OutputSoftmax"]),
            },
            "recurrent": {
                "Forget": np.zeros_like(self.parameters["recurrent"]["Forget"]),
                "Input": np.zeros_like(self.parameters["recurrent"]["Input"]),
                "Candidate": np.zeros_like(self.parameters["recurrent"]["Candidate"]),
                "Output": np.zeros_like(self.parameters["recurrent"]["Output"]),
            }
#             "bias": {
#                 "b_Forget": np.zeros_like(self.parameters["bias"]["Forget"]),
#                 "b_Input": np.zeros_like(self.parameters["bias"]["Input"]),
#                 "b_Candidate": np.zeros_like(self.parameters["bias"]["Candidate"]),
#                 "b_Output": np.zeros_like(self.parameters["bias"]["Output"]),
#                 "b_OutputSoftmax": np.zeros_like(self.parameters["bias"]["OutputSoftmax"]),
#             }
        }

        # Set the next cell and hidden state equal to zero
#         print(f"==============PERFORMING BACKWARD===========\n")
#         print(f"INPUTS:\nforward_pass: \n{forward_pass}\ntargets: \n{targets}\nprediction: \n{prediction}")
#         print(f"\n==============END OF INPUTS=================\n")
        
#         print("\n===============PROCESSING------------------\n")
#         next_stm = np.zeros_like(forward_pass["next_stm"])  # h
#         next_ltm = np.zeros_like(forward_pass["next_ltm"])  # C
        next_ltm = forward_pass["next_ltm"]
        next_stm = forward_pass['next_stm']

        loss = 0
        # Compute the cross entropy
#         for t in reversed(range(self.hidden_size)):
        loss += self.calculate_loss(prediction, targets)
#         print(f"[{t}] LOSS:  {loss}")


        # Compute the derivative of the relation of the hidden-state to the output gate
        dv = np.copy(prediction)
        dv -= 1
#         print(f"dv (OUTPUT_GATE):  \n{dv}\n")
#         print(f"gradients[OutputSoftmax]:  \n{gradients['weights']['OutputSoftmax']}\n")
#         print(f"next_stm.T: \n{next_stm}")
#         print(f"np.dot(next_stm.T, dv): {np.dot(dv, next_stm)}")
        gradients["weights"]["OutputSoftmax"] += np.dot(dv, next_stm).T
        #gradients["bias"]["b_OutputSoftmax"] += dv.T 


        # Compute the derivative of the hidden state and output gate
        dh = np.dot(self.parameters["weights"]["OutputSoftmax"], dv).T
#         print("next_stm: ",next_stm)
#         print(f"dh:  \n{dh.T}\n")
        dh += next_stm


#         print(f"dh: {dh}")
#         print(f"next_ltm: {next_ltm}")
        do = dh * np.tanh(next_ltm)
#         print(f"do: {do}")
#         print(f"w_output: {forward_pass['Output']}")
        do = sigmoid(forward_pass["Output"]) * do
#         print(f"[{t}] do: \n{do}\n")
        # Update the gradients with respect to the output gate
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
#         print(f"do: {do.T.shape}")
#         print(f"Concat_input: {forward_pass['Concat_Input'].shape}")
        gradients["weights"]["Output"] += np.dot(do.T, forward_pass["Concat_Input"]).T
        gradients["recurrent"]["Output"] += np.dot(do.T, next_stm)

        #gradients["bias"]["b_Output"] += do
#         print(f"[{t}] gradients[Output] AFTER UPDATE:  \n{gradients['weights']['Output']}\n")


        # Compute the derivative of the cell state and candidate g
        dC = np.copy(next_ltm)
        dC += dh * forward_pass["Output"] * np.tanh(next_ltm)
#         print(f"[{t}] dC:  \n{dC}\n")

        dg = dC * forward_pass["Input"]
        dg = np.tanh(forward_pass["Candidate"]) * dg
#         print(f"[{t}] dg:  \n{dg}\n")


        # Update the gradients with respect to the candidate
#         print(f"[{t}] gradients[Candidate] BEFORE UPDATE:  \n{gradients['weights']['Candidate']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Candidate"] += np.dot(forward_pass["Concat_Input"].T, dg)
        gradients["recurrent"]["Candidate"] += np.dot(dg.T, next_stm)

        #gradients["bias"]["b_Output"] += dg
#         print(f"[{t}] gradients[Candidate] AFTER UPDATE:  \n{gradients['weights']['Candidate']}\n")


        # Compute the derivative of the input gate and update its gradients
        di = dC * forward_pass["Candidate"]
        di = sigmoid(forward_pass["Input"]) * di
#         print(f"[{t}] di:  \n{di}\n")

#         print(f"[{t}] gradients[Input] BEFORE UPDATE:  \n{gradients['weights']['Input']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Input"] += np.dot(forward_pass["Concat_Input"].T, di)
        gradients["recurrent"]["Input"] += np.dot(di.T, next_stm)

        #gradients["bias"]["b_Input"] += di
#         print(f"[{t}] gradients[Input] AFTER UPDATE:  \n{gradients['weights']['Input']}\n")


        # Compute the derivative of the forget gate and update its gradients
        df = dC * next_ltm
        df = sigmoid(forward_pass["Forget"]) * df
#         print(f"[{t}] df:  \n{df}\n")


#         print(f"[{t}] gradients[Forget] BEFORE UPDATE:  \n{gradients['weights']['Forget']}\n")
        # =========================== ?????????DO WE NEED Concat_Input?????????????====================
        gradients["weights"]["Forget"] += np.dot(forward_pass["Concat_Input"].T, df)
        gradients["recurrent"]["Forget"] += np.dot(df.T, next_stm)

        #gradients["bias"]["b_Forget"] += df
#         print(f"[{t}] gradients[Forget] AFTER UPDATE:  \n{gradients['weights']['Forget']}\n")


        # Compute the derivative of the input and update the gradients of the previous hidden and cell state
        dz = (np.dot(df, self.parameters["weights"]["Forget"].T) + np.dot(di, self.parameters["weights"]["Input"].T) + np.dot(
            dg, self.parameters["weights"]["Candidate"].T) + np.dot(do, self.parameters["weights"]["Output"].T))
#         print(f"[{t}] dz:  \n{dz}\n")

        dh_prev = dz[:self.hidden_size, :]
#         print(f"[{t}] dh_prev:  \n{dh_prev}\n")

        dC_prev = forward_pass["Forget"] * dC
#         print(f"[{t}] dC_prev:  \n{dC_prev}\n")


        # Clip gradients
#         print(f"=========\nGRADS BEFORE CLIP:  \n{grads}\n")

        grads = self.__clip_gradient_norm(gradients)

#         print(f"=========\nGRADS AFTER CLIP:  \n{grads}\n")


        return loss, grads

    def __clip_gradient_norm(self, grads, max_norm=0.25):
        """
        Clips gradients to have a maximum norm of `max_norm`.
        This is to prevent the exploding gradients problem.
        """
        # Set the maximum of the norm to be of type float
        max_norm = float(max_norm)
        total_norm = 0
        # Calculate the L2 norm squared for each gradient and add them to the total norm
        for gate, grad in grads["weights"].items():
            grad_norm = np.sum(np.power(grad, 2))
            total_norm += grad_norm
        total_norm = np.sqrt(total_norm)
        # Calculate clipping coeficient
        clip_coef = max_norm / (total_norm + 1e-6)
        # If the total norm is larger than the maximum allowable norm, then clip the gradient
        if clip_coef < 1:
            for gate, grad in grads["weights"].items():
                grad *= clip_coef
        return grads
    
    def Adam(self, parameters, gradients, learning_rate, t, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        Adam optimization algorithm implementation.

        Arguments:
        parameters -- dictionary containing model parameters
        gradients -- dictionary containing gradients of model parameters
        learning_rate -- learning rate for the algorithm
        beta1 -- exponential decay rate for the first moment estimates (default: 0.9)
        beta2 -- exponential decay rate for the second moment estimates (default: 0.999)
        epsilon -- small constant to prevent division by zero (default: 1e-8)

        Returns:
        parameters -- updated model parameters
        """
#         print("===========INSIDE OPTIMIZER INPUTS=========\n")
#         print("parameters: \n", parameters)
#         print("gradients: \n", gradients)

        # Initialize the first and second moment estimates to zero
        first_moment = {}
        second_moment = {}

        # Initialize the parameters with zeros
        for param_name, param in parameters.items():
            first_moment[param_name] = np.zeros_like(param)
            second_moment[param_name] = np.zeros_like(param)

        # Perform Adam update for each parameter
        for param_name, param in parameters.items():
            # Update first moment estimate
            first_moment[param_name] = beta1 * first_moment[param_name] + (1 - beta1) * gradients[param_name]

            # Update second moment estimate
            second_moment[param_name] = beta2 * second_moment[param_name] + (1 - beta2) * np.square(gradients[param_name])

            # Bias correction
            first_moment_corrected = first_moment[param_name] / (1 - np.power(beta1, t))
            second_moment_corrected = second_moment[param_name] / (1 - np.power(beta2, t))

            # Update parameters
            parameters[param_name] -= learning_rate * first_moment_corrected / (np.sqrt(second_moment_corrected) + epsilon)

        return parameters

    
    def update_parameters(self, grads, t=1, lr=0.01):
        # Take a step
        parameters = self.get_parameters
#         print("===========IN UPDATE PARAMETERS========\n")
#         print("grads: ", grads)
#         print("parameters: ", parameters)
        if (self.USE_OPTIMIZER):
            updated_parameters = self.optimizer(parameters=parameters["weights"], gradients=grads["weights"], learning_rate=lr, t=t)
            updated_parameters = self.optimizer(parameters=parameters["recurrent"], gradients=grads["recurrent"], learning_rate=lr, t=t)

        else:
            for (_, parameter), (_, grad) in zip(parameters.items(), grads.items()):
                parameter -= lr * grad

In [ ]:
### """
# TESTING OF FORWARD PASS WITH DUMMY VALUES
# """

# np.random.seed(1337)
# # Example data dimensions
# input_size = 1  # Number of features in the input
# hidden_state = 6  # Number of units in the hidden state/memory cell
# model = my_build_model(hidden_size=hidden_state, input_size = input_size)

# # Generate example input data, previous hidden state, and previous memory cell
# x = np.array([[9]])
# z = np.array([[11]])
# prev_stm = np.zeros((batch_size, hidden_state))
# prev_ltm = np.zeros((batch_size, hidden_state))
# print("\n=========Printing for MY_LSTM===============\n")
# print("INPUTS\n")
# print("(x) Input:\n", x)
# print("(prev_stm) {h} Previous hidden state:\n", prev_stm)
# print("(prev_ltm) {c} Previous memory cell:\n", prev_ltm)
# print("\n=========PERFORM FORWARD PASS==============\n")

# # Perform forward pass
# print("\n=========APPLY SOFTMAX (LAST LAYER OF LSTM)\n")
# forward_pass = model.forward(input_data=x, prev_stm=prev_stm, prev_ltm=prev_ltm)
# forward_pass = model.forward(input_data=z, prev_stm=forward_pass["next_stm"], prev_ltm=forward_pass["next_ltm"])
# reshaped_output = np.array(forward_pass['Output']).reshape(hidden_state)

# # Print the output
# print("===========OUTPUT============================")
# print("(next_ltm) {h} Next hidden state:\n", forward_pass["next_stm"])
# print("(next_stm) {c} Next memory cell:\n", forward_pass["next_ltm"])
# print("(Output_gate) RESHAPED:\n", reshaped_output)
# output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
# output_softmax = softmax(output_softmax.reshape(2,1))
# print(f"softmax(Output)\n {output_softmax}")
# print("\n========END OF PRINTING FOR MY_LSTM========\n")

# print("\n========START OF TENSORFLOW LSTM===========\n")
# model_tf = build_model(x)
# print("Prediction is: ", model_tf.predict(x))
# print("\n========END OF TENSORFLOW LSTM=============\n")


# print("\n\n======START OF BACKWARDPROPAGATION=======\n")
# print(f"softmax(Output)\n {output_softmax}")
# model.backward(forward_pass=forward_pass, prediction=output_softmax, targets=[[1.0],[0.0]])
# print("\n\n======FINISH OF BACKWARDPROPAGATION======\n")

In [ ]:
def build_model(X):
    model = Sequential()
    print(X.shape[1])
    model.add(Embedding(num_words, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

    return model


In [375]:
def train_split(X, Y, test_size=0.8):
    length = int(len(X) * test_size)
    X_train = X[1:length]
    Y_train = Y[1:length]

    X_valid = X[length:]
    Y_valid = Y[length:]
    return X_train, X_valid, Y_train, Y_valid


def my_train(model, X, Y, hidden_size, input_size, num_epochs=2):
    X_train, X_valid, Y_train, Y_valid = train_split(X = X, Y = Y)
    training_loss, validation_loss = [], []

    for i in range(num_epochs):

        epoch_training_loss = 0
        epoch_validation_loss = 0
        
        sentence_size = len(X_train)
        it = 0
        for sentence, targets in zip(X_train, Y_train):

            prev_stm = np.zeros((1, hidden_size))
            prev_ltm = np.zeros((1, hidden_size))

       
            for word in sentence:
                word = np.array([[word]])
                forward_pass = model.forward(word, prev_stm, prev_ltm)
                prev_stm = forward_pass["next_stm"]
                prev_ltm = forward_pass["next_ltm"]
            print("forward_pass['next_stm']", forward_pass["next_stm"])

            output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
            print("=========BEFORE SOFTMAX=========\n", output_softmax)
            print("\n\nWEIGHTS:\n\n", model.get_parameters)
            output_softmax = sigmoid(output_softmax.reshape(1,1))
            loss, grads = model.backward(forward_pass=forward_pass, prediction=output_softmax, targets=targets)
            print("=========AFTER SOFTMAX=========\n", output_softmax)

            params = model.update_parameters(grads=grads)

            epoch_training_loss += loss
            it += 1
#             print(f"Epoch {i+1}, {it} out of {sentence_size} loss: ", loss)


        sentence_size = len(X_valid) 
        it = 0
        for sentence, targets in zip(X_valid, Y_valid):            
            prev_stm = np.zeros((1, hidden_size))
            prev_ltm = np.zeros((1, hidden_size))
    
            for word in sentence:
                word = np.array([[word]])
                forward_pass = model.forward(word, prev_stm, prev_ltm)
                prev_stm = forward_pass["next_stm"]
                prev_ltm = forward_pass["next_ltm"]
            
            print("forward_pass['next_stm']", forward_pass["next_stm"])
            output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
            print("=========BEFORE SOFTMAX=========\n", output_softmax)
            output_softmax = sigmoid(output_softmax.reshape(1,1))
            print("=========AFTER SOFTMAX=========\n", output_softmax)

            
            loss = model.calculate_loss(output_softmax, targets)

            # Update loss
            epoch_validation_loss += loss
            it += 1
#             print(f"Epoch {i}, {it} out of {sentence_size} loss: ", loss)



        # Save loss for plot
        training_loss.append(epoch_training_loss / len(X_train))
        validation_loss.append(epoch_validation_loss / len(X_valid))

        # Print loss every 2 epochs
        import statistics
        print(f'Epoch {i+1}, training loss: {statistics.mean(training_loss)}, validation loss: {statistics.mean(validation_loss)}')
        print(f'sentence sentence {i}:')
        print(sentence)

        print(f'\nTarget sequence {i}:')
        print(targets)

        print('\nPredicted sequence:')
        output_softmax = np.dot(forward_pass["next_stm"], model.parameters["weights"]["OutputSoftmax"]) + model.parameters["bias"]["OutputSoftmax"]
        output_softmax = softmax(output_softmax.reshape(1,1))

        print(output_softmax)
    return training_loss, validation_loss

In [ ]:
"""
====================FINAL=====================
Preparing the dataset
"""

def convert(x):
    """
    Coverting JSON to pandas dataframe

    """    
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob



def filter_data(data):
    """
    Converting into pandas dataframe and filtering only text and ratings given by the users
    """

    df = pd.DataFrame([convert(line) for line in data])
    df.drop(columns=df.columns.difference(['text','stars']),inplace=True)
    df.loc[:, ("sentiment")] = 0
    

#I have considered a rating above 3 as positive and less than or equal to 3 as negative.
    df.loc[:,'sentiment']=['pos' if (x>3) else 'neg' for x in df.loc[:, 'stars']]
    df.loc[:,'text'] = df.loc[:,'text'].apply(lambda x: x.lower())
    df.loc[:,'text'] = df.loc[:,'text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    for idx,row in df.iterrows():
        df.loc[:,'text']= [x for x in df.loc[:,'text']]
    return df

def read_data():
    json_filename = 'review_mockup_500.json'
    with open(json_filename,'rb') as f:
        data = f.readlines()
    data = filter_data(data)
    tokenizer = Tokenizer(num_words = num_words, split=' ')
    tokenizer.fit_on_texts(data.loc[:,'text'].values)
    X = tokenizer.texts_to_sequences(data.loc[:,'text'].values)
    Y = pd.get_dummies(data['sentiment'], dtype=int).values[:, 0]   
    return X, Y

X, Y = read_data()
print(Y)

In [378]:
np.random.seed(1337)
batch_size = 1  # Number of training examples
input_size = 1  # Number of features in the input
hidden_size = 4  # Number of units in the hidden state/memory cell
model = MY_LSTM.my_build_model(hidden_size=hidden_size, input_size=input_size)
prev_stm = np.zeros((batch_size, hidden_size))
prev_ltm = np.zeros((batch_size, hidden_size))

my_train(model=model, X=X, Y=Y, hidden_size=hidden_size, input_size=input_size, num_epochs=5)

forward_pass['next_stm'] [[-1.0342625e-93 -0.0000000e+00 -0.0000000e+00  3.3498763e-42]]
=========BEFORE SOFTMAX=========
 [[7.74316602e-43]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.57227201, -0.1271049 , -0.45050259, -0.39882011],
       [ 0.54464203,  0.20262196, -0.16629484, -0.79068155],
       [-0.15363508, -0.02857274,  0.8752375 , -0.33556401],
       [ 0.57210527, -0.54877868, -0.04858248,  0.19271741],
       [ 0.15799251,  0.80050874, -0.03150396,  0.25694745]]), 'Input': array([[ 0.25930551,  0.02282924, -0.5414453 , -0.76773484],
       [ 0.11518677,  0.41167588, -0.62490672,  0.3278555 ],
       [ 0.03892055,  0.38380293, -0.23356615,  0.41815967],
       [-0.89055101, -0.23739802, -0.38321826, -0.02099871],
       [ 0.35340735, -0.791416  , -0.33899734,  0.35748501]]), 'Candidate': array([[-0.43176156, -0.1792191 ,  0.36290298, -0.49477758],
       [ 0.50085441,  0.69860783, -0.09941551, -0.42837444],
       [ 0.49162104, -0.65076383, -0.20646009, -0.52256074],
   

forward_pass['next_stm'] [[-4.27058482e-015  1.06500189e-212 -3.05548939e-271 -5.22304694e-053]]
=========BEFORE SOFTMAX=========
 [[1.60787255e-15]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.27252177,  0.06983603, -0.45049645, -0.09886959],
       [ 0.46201899,  0.14679573, -0.16629528, -0.86836321],
       [-0.03257938,  0.06432112,  0.87524208, -0.20841147],
       [ 0.67135242, -0.3817561 , -0.0485732 ,  0.29273308],
       [-0.14200634,  0.50070108, -0.03519071, -0.04305253]]), 'Input': array([[ 0.5583143 , -0.14085611, -0.51723651, -0.46848124],
       [ 0.01680375,  0.50238869, -0.62706651,  0.22979681],
       [ 0.14075103,  0.28479262, -0.21498039,  0.52030967],
       [-0.79066433, -0.48456873, -0.35137852,  0.07933028],
       [ 0.05340825, -0.6914008 , -0.49435696,  0.05748516]]), 'Candidate': array([[-0.1327133 , -0.1792191 ,  0.36290298, -0.19549425],
       [ 0.40252474,  0.69860783, -0.09941551, -0.52637514],
       [ 0.59352778, -0.65076383, -0.20646009, -0.420320

forward_pass['next_stm'] [[-1.56826972e-06 -6.65440548e-26 -6.13393328e-34 -1.22034382e-12]]
=========BEFORE SOFTMAX=========
 [[8.96442276e-07]]


WEIGHTS:

 {'weights': {'Forget': array([[-1.72522911e-01,  2.69580277e-01, -3.50526721e-01,
         1.12928228e-03],
       [ 4.62018991e-01,  1.70629311e-01, -1.66295279e-01,
        -8.68363209e-01],
       [-3.25793763e-02,  6.45122534e-02,  8.75242077e-01,
        -2.08411469e-01],
       [ 6.77993708e-01, -2.87068153e-01, -4.83053676e-02,
         2.99468700e-01],
       [-4.04435309e-01,  7.12199320e-04, -1.35190278e-01,
        -3.16826832e-01]]), 'Input': array([[ 0.65831383,  0.05909262, -0.41723944, -0.36848176],
       [ 0.01680375,  0.5698449 , -0.62706651,  0.22979681],
       [ 0.14075103,  0.28604525, -0.21498039,  0.52030967],
       [-0.77598001, -0.38502422, -0.34867438,  0.09286531],
       [-0.24658795, -0.99140096, -0.59435692, -0.2425119 ]]), 'Candidate': array([[-0.03271417, -0.07943534,  0.46287853, -0.09549514],
 

forward_pass['next_stm'] [[-1.03177632e-113 -2.01131323e-310 -1.15602419e-317 -7.19329524e-113]]
=========BEFORE SOFTMAX=========
 [[2.34303506e-113]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.17252284,  0.26979306, -0.35052672,  0.00112935],
       [ 0.46201899,  0.17062931, -0.16629528, -0.86836321],
       [-0.03257938,  0.06451225,  0.87524208, -0.20841147],
       [ 0.67799373, -0.28706815, -0.04830537,  0.29946872],
       [-0.50795549, -0.09928482, -0.22865891, -0.42033674]]), 'Input': array([[ 0.67497238,  0.09963816, -0.41712958, -0.35445236],
       [ 0.01680375,  0.5698449 , -0.62706651,  0.22979681],
       [ 0.14075103,  0.28604525, -0.21498039,  0.52030967],
       [-0.7710661 , -0.38493639, -0.34864584,  0.09691165],
       [-0.54655637, -1.28891955, -0.79190418, -0.54247681]]), 'Candidate': array([[-0.03271406, -0.07943534,  0.46287853, -0.09549504],
       [ 0.40252474,  0.69860783, -0.09941551, -0.52637514],
       [ 0.59352778, -0.65076383, -0.20646009, -0.42032

forward_pass['next_stm'] [[-3.25806826e-29 -5.84412578e-75 -2.71349144e-69 -5.20727187e-29]]
=========BEFORE SOFTMAX=========
 [[3.88039066e-29]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.17252284,  0.26979306, -0.35052672,  0.00112935],
       [ 0.46201899,  0.17062931, -0.16629528, -0.86836321],
       [-0.03257938,  0.06451225,  0.87524208, -0.20841147],
       [ 0.67799373, -0.28706815, -0.04830537,  0.29946872],
       [-0.50795549, -0.09928482, -0.22865891, -0.42033674]]), 'Input': array([[ 0.67497238,  0.09963816, -0.41712958, -0.35445236],
       [ 0.01680375,  0.5698449 , -0.62706651,  0.22979681],
       [ 0.14075103,  0.28604525, -0.21498039,  0.52030967],
       [-0.7710661 , -0.38493639, -0.34864584,  0.09691165],
       [-0.54661528, -1.28891955, -0.7919042 , -0.54253217]]), 'Candidate': array([[-0.03271406, -0.07943534,  0.46287853, -0.09549504],
       [ 0.40252474,  0.69860783, -0.09941551, -0.52637514],
       [ 0.59352778, -0.65076383, -0.20646009, -0.42032087],

...

forward_pass['next_stm'] [[-0. -0. -0. -0.]]
=========BEFORE SOFTMAX=========
 [[0.]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.05039074,  0.36379769, -0.23967133,  0.11681777],
       [ 0.46201901,  0.17072942, -0.16629525, -0.8683632 ],
       [-0.03246218,  0.0660114 ,  0.87531302, -0.20834907],
       [ 0.79877287, -0.18993055,  0.06187072,  0.41420058],
       [-1.17804318, -0.76733407, -0.82882583, -1.07415558]]), 'Input': array([[ 0.85235714,  0.42371814, -0.18200394, -0.19080545],
       [ 0.01680382,  0.66296565, -0.61371027,  0.22979729],
       [ 0.14113698,  0.39641489, -0.14463755,  0.52052165],
       [-0.59672186, -0.0721764 , -0.12180384,  0.25764601],
       [-1.98999226, -2.74670042, -2.28036274, -1.99471505]]), 'Candidate': array([[ 0.09599462,  0.01531719,  0.57722125,  0.02604058],
       [ 0.40252477,  0.69860784, -0.0994155 , -0.52637513],
       [ 0.59369545, -0.6507611 , -0.20636401, -0.42022684],
       [ 0.74940926, -0.13435591,  0.30289858,  0.76904622]

forward_pass['next_stm'] [[-9.69753821e-20 -1.36374858e-51 -2.66566914e-29 -8.74052969e-20]]
=========BEFORE SOFTMAX=========
 [[3.40412511e-19]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.05039074,  0.36379769, -0.23967133,  0.11681777],
       [ 0.46201901,  0.17072942, -0.16629525, -0.8683632 ],
       [-0.03246218,  0.0660114 ,  0.87531302, -0.20834907],
       [ 0.79877287, -0.18993055,  0.06187072,  0.41420058],
       [-1.17804318, -0.76733407, -0.82882583, -1.07415558]]), 'Input': array([[ 0.85235714,  0.42371814, -0.18200394, -0.19080545],
       [ 0.01680382,  0.66296565, -0.61371027,  0.22979729],
       [ 0.14113698,  0.39641489, -0.14463755,  0.52052165],
       [-0.59672186, -0.0721764 , -0.12180384,  0.25764601],
       [-1.98999226, -2.74670043, -2.28036275, -1.99471505]]), 'Candidate': array([[ 0.09599462,  0.01531719,  0.57722125,  0.02604058],
       [ 0.40252477,  0.69860784, -0.0994155 , -0.52637513],
       [ 0.59369545, -0.6507611 , -0.20636401, -0.42022684],

forward_pass['next_stm'] [[-6.68667858e-276 -0.00000000e+000 -0.00000000e+000 -1.95805140e-253]]
=========BEFORE SOFTMAX=========
 [[2.92282661e-253]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.05039074,  0.36379769, -0.23967133,  0.11681777],
       [ 0.46201901,  0.17072942, -0.16629525, -0.8683632 ],
       [-0.03246218,  0.0660114 ,  0.87531302, -0.20834907],
       [ 0.79877287, -0.18993055,  0.06187072,  0.41420058],
       [-1.17804318, -0.76733407, -0.82882583, -1.07415558]]), 'Input': array([[ 0.85235714,  0.42371814, -0.18200394, -0.19080545],
       [ 0.01680382,  0.66296565, -0.61371027,  0.22979729],
       [ 0.14113698,  0.39641489, -0.14463755,  0.52052165],
       [-0.59672186, -0.0721764 , -0.12180384,  0.25764601],
       [-1.98999226, -2.74670043, -2.28036275, -1.99471505]]), 'Candidate': array([[ 0.09599462,  0.01531719,  0.57722125,  0.02604058],
       [ 0.40252477,  0.69860784, -0.0994155 , -0.52637513],
       [ 0.59369545, -0.6507611 , -0.20636401, -0.42022

forward_pass['next_stm'] [[-1.63684434e-25 -8.66545374e-65 -2.04645362e-32 -1.01971314e-23]]
=========BEFORE SOFTMAX=========
 [[1.55758417e-23]]


WEIGHTS:

 {'weights': {'Forget': array([[-0.05039074,  0.36379769, -0.23967133,  0.11681777],
       [ 0.46201901,  0.17072942, -0.16629525, -0.8683632 ],
       [-0.03246218,  0.0660114 ,  0.87531302, -0.20834907],
       [ 0.79877287, -0.18993055,  0.06187072,  0.41420058],
       [-1.17804318, -0.76733407, -0.82882583, -1.07415558]]), 'Input': array([[ 0.85235714,  0.42371814, -0.18200394, -0.19080545],
       [ 0.01680382,  0.66296565, -0.61371027,  0.22979729],
       [ 0.14113698,  0.39641489, -0.14463755,  0.52052165],
       [-0.59672186, -0.0721764 , -0.12180384,  0.25764601],
       [-1.98999226, -2.74670043, -2.28036275, -1.99471505]]), 'Candidate': array([[ 0.09599462,  0.01531719,  0.57722125,  0.02604058],
       [ 0.40252477,  0.69860784, -0.0994155 , -0.52637513],
       [ 0.59369545, -0.6507611 , -0.20636401, -0.42022684],

KeyboardInterrupt: 